In [1]:
import argparse

from utils.logger import get_logger
from utils.arg_parser import Argments
from loader.data_loader import GeneralDataLoaderCls, NbsDataLoaderCls,\
                               GeneralDataLoaderSeg, NbsDataLoaderSeg,NbsDataLoaderRgs
from runners.cnn_runner import CnnRunner
from runners.nbs_runner import NbsRunner
from runners.mcd_runner import McdRunner

import torch
print(torch.cuda.is_available())

import sys
print(sys.executable)


True
c:\Users\user\.conda\envs\torch\python.exe


In [2]:
# making instance
argparser = argparse.ArgumentParser()
# adding the values
argparser.add_argument("yaml")
argparser.add_argument("--phase", default="train", type=str) # train / infer 
argparser.add_argument("--index", default=-1, type=int) # meaning ? -> used in file name(spcific version : file1,file2,...) -1 means nothing is added in file name.
argparser.add_argument("--gpus", default="0", type=str)
argparser.add_argument("--local_rank", default=0, type=int) # meaning ? logger level?
# save 
cmd_args = argparser.parse_args(["example"])

arg = Argments(f"scripts/{cmd_args.yaml}.yaml", cmd_args) # stll many funcs to read
setup = arg['setup']
model_path = arg['path/model_path']
logger = get_logger(f"{model_path}/log.txt")



setting the python env
cmd_args.local_rank:  0


In [3]:
if setup['rank'] == 0:
    logger.info(arg)

model_type = setup['model_type']
dataset = arg['path/dataset']
is_seg = False

2023-05-12 16:20:19,973 - outs/nbs/Reg_model/custom_train/log.txt - INFO - scripts/example.yaml
{'path': {'dataset': 'custom', 'postfix': 'train', 'model_path': 'outs/nbs/Reg_model/custom_train'}, 'setup': {'model_type': 'nbs', 'batch_size': 20, 'cpus': 1, 'gpus': '0', 'seed': 0, 'num_epoch': 2, 'phase': 'train', 'epoch_th': 0, 'num_mc': 5, 'n_a': 2, 'adv_training': False, 'index': -1, 'local_rank': 0, 'rank': 0, 'dist_size': 0}, 'module': {'model': {'name': 'Reg_model', 'num_classes': 1, 'dropout_rate': 0.0}, 'optim': {'name': 'SGD', 'lr': 0.05, 'momentum': 0.9, 'nesterov': True, 'weight_decay': 0.001}, 'lr_scheduler': {'name': 'CosineAnnealingLR', 'T_max': 200}, 'loss': ['NbsLoss'], 'loss_args': {'NbsLoss': {'reduction': 'mean'}}, 'loss_weight': [1.0], 'val_metric': 'Accuracy', 'test_metric': 'Accuracy', 'metric_args': {'nlabels': 10, 'reduction': 'none'}}}


In [4]:
# from torchvision.datasets import MNIST
# trainset = MNIST(root='.mnist', train=True, download=True)

In [19]:
# from loader.data_loader import CustomDataset
# def _load_custom():
#     n_train= 10000
#     n_test = 1000
#     # true beta
#     beta = torch.ones([28 * 28 * 3,1])
#     # mean 5, std 2
#     train_X = torch.normal(5,2,size=(n_train,28 * 28 * 3))
#     train_y = torch.mm(train_X,beta)
#     test_X = torch.normal(5,2,size=(n_test,28 * 28 * 3))
#     test_y = torch.mm(test_X,beta)
#     trainset = CustomDataset(train_X,train_y)
#     testset = CustomDataset(test_X,test_y)
#     print(train_X.size())
#     print(train_y.size())
#     return {'train': trainset, 'test': testset}


# dataset = _load_custom()
# trainset = dataset["train"]
# testset = dataset["test"]

# import random
# def _get_split_indices_rgs(trainset, p, seed):
#     length = len(trainset)
#     indices = list(range(length))
#     random.Random(seed).shuffle(indices)
#     sep = int(length * p)
#     return indices[sep:], indices[:sep]
# _get_split_indices_rgs(trainset,0.1,0)

torch.Size([10000, 2352])
torch.Size([10000, 1])


([2599,
  9079,
  9548,
  1860,
  6480,
  6912,
  1955,
  2774,
  1579,
  9724,
  5150,
  4154,
  5644,
  6687,
  6952,
  1170,
  5494,
  7044,
  5373,
  8941,
  8246,
  9933,
  8285,
  3017,
  1614,
  9529,
  5694,
  1554,
  1959,
  1445,
  3328,
  8181,
  8471,
  3886,
  6704,
  8493,
  8953,
  1028,
  6602,
  6986,
  5505,
  4472,
  6236,
  4133,
  1668,
  416,
  4122,
  344,
  907,
  7011,
  9214,
  4688,
  5861,
  8985,
  5594,
  8722,
  4889,
  2549,
  732,
  7956,
  579,
  3238,
  3880,
  289,
  9014,
  5406,
  6750,
  2744,
  6702,
  1338,
  8095,
  8391,
  3344,
  5923,
  2457,
  694,
  1086,
  4754,
  3232,
  9837,
  8052,
  2832,
  3286,
  4879,
  8088,
  989,
  5627,
  8523,
  4076,
  82,
  4493,
  1002,
  9423,
  359,
  1967,
  8731,
  1301,
  8325,
  1021,
  4842,
  8971,
  1623,
  2577,
  8249,
  1589,
  1558,
  4729,
  3193,
  7408,
  8002,
  8127,
  3661,
  1093,
  2608,
  7746,
  9218,
  3767,
  6229,
  2062,
  144,
  3702,
  9426,
  7892,
  161,
  562,
  394,
  3007,

In [4]:
if 'nbs' in model_type: # added in n_a (comparing the else loop)
    _data_loader = NbsDataLoaderRgs # n_a exists
    data_loader = _data_loader(dataset, setup['batch_size'],
                                setup['n_a'], setup['cpus'], setup['seed'])
    runner = NbsRunner(data_loader, **arg.module, num_epoch=setup['num_epoch'], # core part 
                        logger=logger, model_path=model_path, rank=setup['rank'],
                        epoch_th=setup['epoch_th'], num_mc=setup['num_mc'],
                        adv_training=setup['adv_training'])

2023-05-12 16:20:22,696 - outs/nbs/Reg_model/custom_train/log.txt - DEBUG - outs\nbs\Reg_model\custom_train
2023-05-12 16:20:22,696 - outs/nbs/Reg_model/custom_train/log.txt - DEBUG - Failed to load, not existing file


torch.Size([10000, 2352])
torch.Size([10000, 1])
here1
here3
here4
here2


In [5]:
def train(self):
    self.log("Start to train", 'debug')
    for epoch in range(self.epoch, self.num_epoch):
        self.model.train()
        loader = self.loader.load("train")
        if self.rank == 0:
            t_iter = tqdm(loader, total=self.loader.len,
                            desc=f"[Train {epoch}]")
        else:
            t_iter = loader
        losses = 0
        times = []
        for i, batch in enumerate(t_iter):
            t = time()
            loss = self._train_a_batch(batch) # update the params.
            times += [time() - t]
            losses += loss
            if self.rank == 0:
                t_iter.set_postfix(loss=f"{loss:.4} / {losses/(i+1):.4}")

        self.log(f"[Train] epoch:{epoch} loss:{losses/(i+1)}", 'info')
        print("Batch Training Time : ", np.mean(times))
        self.lr_scheduler.step()
        self.val(epoch)

In [6]:
if setup['phase'] == 'train':
    runner.train() 
    runner.test(is_seg) 
elif setup['phase'] == 'test':
    runner.test(is_seg)


if __name__ == "__main__":
    main()

2023-05-12 16:20:26,126 - outs/nbs/Reg_model/custom_train/log.txt - DEBUG - Start to train
[Train 0]:   0%|          | 0/450 [00:02<?, ?it/s]


TypeError: empty(): argument 'size' must be tuple of ints, but found element of type Tensor at pos 1